# Import Libraries and LOWESS Utils

In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from moepy import lowess

# Read Data

In [25]:
data = pd.read_csv('../res/model_data/batemo_model_data.csv')
data = data[data.V >= 2.5]

# Train LOWESS Model

In [ ]:
min_SOH, max_SOH = data.SOH.min(), data.SOH.max()
min_I, max_I = int(data.I.min()), int(data.I.max())

# Initialize lists to store results
V_acc = []
SOC_pred = []

for i in range(min_SOH, max_SOH + 1):
    for j in range(min_I, max_I + 1):
        data_ij = data[(data.SOH == i) & (data.I == j)]
        V = np.array(data_ij.V)
        SOC = np.array(data_ij.SOC)
        lowess_model = lowess.Lowess()
        lowess_model.fit(V, SOC, frac=0.01)
        y_pred = lowess_model.predict(V)
        
        # Append results to lists
        V_acc.extend(V)
        SOC_pred.extend(y_pred)

# Export the results to a new CSV
results = pd.DataFrame({'V': V_acc, 'SOC': SOC_pred})
results.to_csv('../res/model_data/lowess_results.csv', index=False)

# Plot LOWESS Model
# plt.plot(V, y_pred, '--', label='Estimate', color='k', zorder=3)
# plt.scatter(V, SOC, label='With Noise', color='C1', s=5, zorder=1)
# plt.legend(frameon=False)

C:\Users\LucasTroy\AppData\Local\Programs\Python\Python311\Lib\site-packages\moepy\lowess.py:38: RuntimeWarning: divide by zero encountered in divide
  dist_to_weights = lambda dist, dist_threshold=1: (1 - ((np.abs(dist)/dist_threshold).clip(0, 1) ** 3)) ** 3
C:\Users\LucasTroy\AppData\Local\Programs\Python\Python311\Lib\site-packages\moepy\lowess.py:38: RuntimeWarning: invalid value encountered in divide
  dist_to_weights = lambda dist, dist_threshold=1: (1 - ((np.abs(dist)/dist_threshold).clip(0, 1) ** 3)) ** 3


In [2]:
# TODO: look up how to reference the csv given bus values for V and I as well as SOH from model for predicting SOC

# for i in range(0, 101):
#     for j in range(-55, 56):

optimal_k = []

for i in range(90,93):
    for j in range(10,11):

        data = data[data.SOH == i]
        data = data[data.I == j]

        colA = np.array(data.V)
        colB = np.array(data.SOC)

        mcolA = np.mat(colA)
        mcolB = np.mat(colB)

        m = np.shape(mcolB)[1]
        one = np.ones((1, m), dtype = int)

        # horizontal stacking
        X = np.hstack((one.T, mcolA.T))
        print(X.shape)

        # ypred = localWeightRegression(X, mcolB, 0.02)

        # find the variance of ypred and colB looping through different k values
        # find the optimal k value that minimizes the variance

        # Initialize lists to store k values and corresponding variances
        k_values = []
        variances = []

        # Range of k values to try
        k_range = np.linspace(0.01, 0.1, 10)  # Adjust as needed

        for k in k_range:
            ypred = localWeightRegression(X, mcolB, k)
            variance = np.var(ypred - colB)  # Calculate variance between ypred and colB
            k_values.append(k)
            variances.append(variance)

        # Find the index of the minimum variance
        min_variance_index = np.argmin(variances)
        optimal_k.append(k_values[min_variance_index])
        # min_variance = variances[min_variance_index]

print(optimal_k)

# plot a histogram of optimal k values
plt.hist(optimal_k, bins=10)
plt.xlabel('k')
plt.ylabel('Frequency')
plt.title('Optimal k Values')
plt.show()

        # print("For i =", i, "and j =", j)
        # print("Optimal k:", optimal_k)
        # print("Min Variance:", min_variance)

        # Plot the variance vs. k values
        # plt.plot(k_values, variances)
        # plt.xlabel('k')
        # plt.ylabel('Variance')
        # plt.title('Variance vs. k')
        # plt.show()

        # append the data to a csv
        # export = pd.DataFrame()
        # export['V'] = colA.round(3)
        # export['I'] = j
        # export['SOH'] = i
        # export['SOC'] = np.round(ypred,3)
        # export.to_csv('new csv.csv', mode='a', header=False)

        # xsort = X.copy()
        # xsort.sort(axis=0)
        # plt.scatter(colA, colB, color='blue')
        # #                V           SOC
        # plt.plot(xsort[:, 1], ypred[X[:, 1].argsort(0)], color='yellow', linewidth=5)
        # plt.xlabel('V')
        # plt.ylabel('SOC')
        # plt.show()



(582, 2)


ValueError: shapes (582,582) and (1,2) not aligned: 582 (dim 1) != 1 (dim 0)

In [ ]:
# sample code for compressing the csv and prepping it to be flashed onto the microcontroller:
import zlib

# Read CSV data
data = pd.read_csv('new csv.csv')

# Convert data to binary format
binary_data = data.to_string().encode('utf-8')

# Compress binary data
compressed_data = zlib.compress(binary_data)

# Write compressed data to file
with open('compressed_data.bin', 'wb') as f:
    f.write(compressed_data)